# Table 2 and 3

This tutorial explains the experimental setup to obtain **one row** of Table 2 or Table 3.

In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC #Linear SVM for feature maps
from sklearn.svm import SVC #Dual SVM for Kernels
from time import time
import numpy as np

'FeatureMaps' and 'DataReader' imports are the modules that contain implementations of the proposed feature maps and the dataset importing procedures.

In [2]:
import FeatureMaps as maps
import DataReader as DR

Here, we give all of of the datasets and their names as commented code lines. A row of Table 2 and Table 3 can be obtained by simply changing the last two lines.

In [3]:
# All datasets
#datasets=[DR.Splice, DR.Wilt,DR.Guide1, DR.Spambase, DR.Phoneme, DR. Magic, DR.Adult ]
# All dataset names
#data_names=['Splice','Wilt', 'Guide 1', 'Spambase', 'Phoneme','Magic','Adult' ]

datasets=[DR.Phoneme]
data_names=['Phoneme']

Following code block creates lists of feature maps, kernels in the order of the columns in Table 1. When a feature map is removed from the list `mapping_functions` then, `p_vals`  list  should be arranged accordingly. Therefore, we suggest the reader to leave this code block as given.

In [4]:
# Set mapping functions
mapping_functions=[maps.linear, maps.phi_p_1,maps.phi_p_1,maps.phi_p_d,maps.phi_p_d]

# Set p values according to the mapping functions in respected order in given Table 1
# p value of the linear kernel is used as a dummy variable and others in respected order in given Table 1
p_vals = [0,1,2,1,2]

# Set mapping function an kernel names  in respected order in given Table 1
kernel_names = ['LIN',r'$\phi_{1,1}$',r'$\phi_{2,1}$',r'$\phi_{1,d}$',r'$\phi_{2,d}$','POL','RBF']

Here, we create parameter sets for kernels and SVM model  along with the random state for splitting datasets.

In [5]:
# Set of POL  kernel parameters
d_params = [2, 3, 4]
# Set  of RBF kernel parameters
g_params = np.power(10.0, range(-5, 5))

# Set of  C parameters
c_params = np.power(10.0, range(-5, 5))

# Random seed for splitting the dataset
random_state=42

We store all of the performance metrics in the  dictionary named 'results' in order to print the obtained results in a readable format.  The metrics are stored by regarding the dataset-feature map and kernel orders, which are assigned above.  

In [6]:
# A dictionary to store  all results
results={'Dataset':[], 'Training Acc.':[], 'Test Acc.':[], 'Training Time':[]}

We perform grid search with stratified two-fold cross validation on each training part to find the best performing hyperparameters. Here, we also set the cross validation object to make sure each feature map and kernel use  same training and test parts.

In [7]:
for i in range(len(datasets)):
    results['Dataset'].append(data_names[i])
    X_train, X_test, y_train, y_test= datasets[i]()
    #set dictionaries to store performance metrics
    over_all_performance_metrics= {'Kernel': [], 'Training Acc': [], 'Test Acc': [], 'Training Time': []}
    #2-fold cross validation object
    inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=random_state)

Following code block performs grid search with stratified two-fold cross validation  to find the best  performing hyperparameters for the linear and the proposed feature maps.

In [8]:
#suppress the convergence warning for LinearSVC
import warnings
warnings.filterwarnings("ignore")

for m in range(len(mapping_functions)):
    over_all_performance_metrics['Kernel'].append(kernel_names[m])
    acc_by_param = []
    #begin: grid search 
    for ci in c_params:
        all_acc = []
       #begin: two-fold
        for inner_train_index, inner_test_index in inner_cv.split(X_train, y_train):
            #begin: scaling
            scaler = StandardScaler()
            X_inner_train = scaler.fit_transform(X_train[inner_train_index])
            X_inner_test = scaler.transform(X_train[inner_test_index])
            #end: scaling
            y_inner_train, y_inner_test = y_train[inner_train_index], y_train[inner_test_index]
            clf = LinearSVC(C=ci, dual=False).fit(mapping_functions[m](X_inner_train, p=p_vals[m]), y_inner_train)
            all_acc.append(accuracy_score(y_inner_test, clf.predict(mapping_functions[m](X_inner_test, p=p_vals[m]))))
        #end: two-fold
        acc_by_param.append(np.mean(all_acc))
    #end: grid search 
    #get best hyperparameters
    best_c = c_params[np.argmax(acc_by_param)]
    scaler = StandardScaler()
    #being: scaling
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    #end: scaling
    s = time()
    XD_train = mapping_functions[m](X_train_scaled,p=p_vals[m])
    clf = LinearSVC(C=best_c, dual=False).fit(XD_train, y_train)
    over_all_performance_metrics['Training Time'].append( round(time() - s, 4))
    over_all_performance_metrics['Training Acc'].append(round(100*accuracy_score(y_train, clf.predict(XD_train)),2))
    over_all_performance_metrics['Test Acc'].append(round(100*accuracy_score(y_test, clf.predict(mapping_functions[m](X_test_scaled, p=p_vals[m]))),2))

Following code block, performs grid search with stratified two-fold cross validation  to find the best  performing hyperparameters for the POL kernel.

In [9]:
over_all_performance_metrics['Kernel'].append('POL')
acc_by_param = {'ci': [], 'di': [], 'inner_test_acc': []}
#begin: grid search
for ci in c_params:
    for di in d_params:
        all_acc = []
        #begin: two-fold
        for inner_train_index, inner_test_index in inner_cv.split(X_train, y_train):
            #begin: scaling
            scaler = StandardScaler()
            X_inner_train = scaler.fit_transform(X_train[inner_train_index])
            X_inner_test = scaler.transform(X_train[inner_test_index])
            #end:  scaling
            y_inner_train, y_inner_test = y_train[inner_train_index], y_train[inner_test_index]
            clf = SVC(kernel='poly', C=ci, degree=di).fit(X_inner_train, y_inner_train)
            all_acc.append(accuracy_score(y_inner_test, clf.predict(X_inner_test)))
        #end: two fold
        acc_by_param['ci'].append(ci)
        acc_by_param['di'].append(di)
        acc_by_param['inner_test_acc'].append(np.mean(all_acc))
#end: grid search 
#get best hyperparameters
best_ind = np.argmax(acc_by_param['inner_test_acc'])
best_c, best_d = acc_by_param['ci'][best_ind], acc_by_param['di'][best_ind]
#begin: scaling
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
#end: scaling
s = time()
clf = SVC(kernel='poly', C=best_c, degree=best_d).fit(X_train_scale, y_train)
over_all_performance_metrics['Training Time'].append(round(time() - s, 4))
over_all_performance_metrics['Training Acc'].append(round(100*accuracy_score(y_train, clf.predict(X_train_scale ),2)))
over_all_performance_metrics['Test Acc'].append(round(100*accuracy_score(y_test, clf.predict(X_test_scale),2)))

Following code block, performs grid search with stratified two-fold cross validation  to find the best  performing hyperparameters for the RBF kernel.

In [10]:
over_all_performance_metrics['Kernel'].append('RBF')
acc_by_param = {'ci': [], 'gi': [], 'inner_test_acc': []}
#begin: grid search
for ci in c_params:
    for gi in g_params:
        all_acc = []
        #begin: two-fold
        for inner_train_index, inner_test_index in inner_cv.split(X_train, y_train):
            #begin: scaling
            scaler = StandardScaler()
            X_inner_train = scaler.fit_transform(X_train[inner_train_index])
            X_inner_test = scaler.transform(X_train[inner_test_index])
            #end: scaling
            y_inner_train, y_inner_test = y_train[inner_train_index], y_train[inner_test_index]
            clf = SVC(kernel='rbf', C=ci, gamma=gi).fit(X_inner_train, y_inner_train)
            all_acc.append(accuracy_score(y_inner_test, clf.predict(X_inner_test)))
        #end: two-fold
        acc_by_param['ci'].append(ci)
        acc_by_param['gi'].append(gi)
        acc_by_param['inner_test_acc'].append(np.mean(all_acc))
#end: grid search 
#get best hyperparameters
best_ind = np.argmax(acc_by_param['inner_test_acc'])
best_c, best_g = acc_by_param['ci'][best_ind], acc_by_param['gi'][best_ind]
#begin: scaling
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
#end: scaling
s = time()
clf = SVC(kernel='rbf', C=best_c, gamma=best_g).fit(X_train_scale, y_train)
over_all_performance_metrics['Training Time'].append(round(time() - s, 4))
over_all_performance_metrics['Training Acc'].append(round(100 * accuracy_score(y_train, clf.predict(X_train_scale), 2)))
over_all_performance_metrics['Test Acc'].append(round(100 * accuracy_score(y_test, clf.predict(X_test_scale), 2)))
results['Training Time'].append(over_all_performance_metrics['Training Time'])
results['Training Acc.'].append(over_all_performance_metrics['Training Acc'])
results['Test Acc.'].append(over_all_performance_metrics['Test Acc'])

Following code block prints the performance metrics in a readeable format.

In [11]:
print('***** Table 2: Test Accuracies')
n_datasets=len(datasets)
n_kenels=len(kernel_names)
for d in range(n_datasets):
    print()
    print('***'+ results['Dataset'][d])
    temp=''
    for k in range(n_kenels):
        temp+=kernel_names[k] +'\t'+ str(results['Test Acc.'][d][k])+ '\t'+ '\t'
    print(temp)

***** Table 2: Test Accuracies

***Phoneme
LIN	75.46		$\phi_{1,1}$	73.61		$\phi_{2,1}$	74.29		$\phi_{1,d}$	76.82		$\phi_{2,d}$	76.57		POL	78.0		RBF	88.0		


In [12]:
print('***** Table 3: Training Times in Seconds')
n_datasets=len(datasets)
n_kenels=len(kernel_names)
for d in range(n_datasets):
    print()
    print('***'+ results['Dataset'][d])
    temp=''
    for k in range(n_kenels):
        temp+=kernel_names[k] +'\t'+ str(results['Training Time'][d][k])+ '\t'+ '\t'

    print(temp)

***** Table 3: Training Times in Seconds

***Phoneme
LIN	0.0023		$\phi_{1,1}$	0.0028		$\phi_{2,1}$	0.0026		$\phi_{1,d}$	0.0053		$\phi_{2,d}$	0.0041		POL	143.6466		RBF	0.3508		
